In [1]:
import pandas as pd
import os
!pip install pykrige

Defaulting to user installation because normal site-packages is not writeable


wb = pd.read_csv("/home/sysadm/Downloads/sentinal5p_So2/Wb_2022_3_Final_So2_csv")

delhi

delhi.columns.tolist()

date_column = delhi.columns.tolist()[2:]
date_column

df1

# Create a DataFrame with only latitude, longitude, and the specified date column
df1 = delhi[['latitude', 'longitude', '2023-09-21']]

# Create a DataFrame with latitude and longitude where the specified date column's values are NaN
nan_locations_df = df1[df1['2023-09-21'].isna()][['latitude', 'longitude']]
nan_locations_df

# Create a DataFrame with latitude, longitude, and the specified date column excluding NaN values
pollutant_df = df1.dropna(subset=['2023-09-21'])[['latitude', 'longitude', '2023-09-21']]
pollutant_df

In [1]:
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging


# Prepare the data for Ordinary Kriging
lons = np.array(pollutant_df['longitude'])
lats = np.array(pollutant_df['latitude'])
zdata = np.array(pollutant_df['2023-09-21'])

In [31]:
nlags=20

# Perform Ordinary Kriging
OK = OrdinaryKriging(lons, lats, zdata, variogram_model='linear', verbose=True, enable_plotting=False, nlags=nlags)
grid_lon = np.array(nan_locations_df["longitude"])
grid_lat = np.array(nan_locations_df["latitude"])
z1, ss1 = OK.execute('grid', grid_lon, grid_lat)

nan_locations_df['2023-09-21'] = z1[0]
nan_locations_df

# Merge the two DataFrames based on their indices
merged_df = pd.concat([pollutant_df, nan_locations_df], ignore_index=False)
merged_df

# Sort the DataFrame by ascending index
merged_df = merged_df.sort_index(ascending=True)
merged_df

# Get the list of columns that have NaN values
columns_with_nan = delhi.columns[delhi.isna().any()].tolist()
columns_with_nan

In [4]:
import pandas as pd
from pykrige.ok import OrdinaryKriging
import numpy as np

def estimate_pollutant(df, date_column, nlags=20):
    # Create a DataFrame with only latitude, longitude, and the specified date column
    df1 = df[['longitude', 'latitude', date_column]]

    # Create a DataFrame with latitude and longitude where the specified date column's values are NaN
    nan_locations_df = df1[df1[date_column].isna()][['longitude', 'latitude']]

    # Create a DataFrame with latitude, longitude, and the specified date column excluding NaN values
    pollutant_df = df1.dropna(subset=[date_column])[['longitude', 'latitude', date_column]]

    # Prepare the data for Ordinary Kriging
    lons = np.array(pollutant_df['longitude'])
    lats = np.array(pollutant_df['latitude'])
    zdata = np.array(pollutant_df[date_column])

    # Perform Ordinary Kriging
    OK = OrdinaryKriging(lons, lats, zdata, variogram_model='linear', verbose=True, enable_plotting=False, nlags=nlags)
    grid_lon = np.array(nan_locations_df["longitude"])
    grid_lat = np.array(nan_locations_df["latitude"])
    z1, ss1 = OK.execute('grid', grid_lon, grid_lat)

    # Fill in NaN values in the specified date column with the interpolated values
    nan_locations_df[date_column] = z1[0]

    # Merge the two DataFrames based on their indices
    merged_df = pd.concat([pollutant_df, nan_locations_df], ignore_index=False)

    # Sort the DataFrame by ascending index
    merged_df = merged_df.sort_index(ascending=True)

    return merged_df

In [5]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_So2/Chosen_state_2023_sep_merge_sentinel5p_So2/Delhi_sep_So2.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_So2/After_krigging_2023_sep_sentinal5p_So2/Delhi_2023_sep_So2_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 6.952165725214201e-08
Nugget: 2.5299323667475754e-07 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 3.5629933595434986e-07
Nugget: 2.855035414613813e-07 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 6.553181026264406e-05
Nugget: 1.2284073907579624e-05 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.7204903552587342e-07
Nugget: 1.1607036206692076e-07 


In [6]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-26,2023-09-27,2023-09-28,2023-09-29
0,28.408461,76.832916,0.000189,-0.000108,-0.000131,0.000181,-0.000238,0.000011,-0.000013,-0.000455,...,-0.000217,0.000365,0.000258,-0.000030,0.000926,0.000014,-0.000789,-0.000536,0.000607,-0.001667
1,28.422062,76.832916,-0.000611,-0.000384,0.000511,-0.001040,0.000123,0.000192,0.000117,-0.000963,...,0.002074,0.000139,0.000050,0.000116,0.000330,0.002956,-0.000165,0.000371,-0.000435,0.000397
2,28.435664,76.832916,-0.000207,-0.000300,0.000162,0.000111,0.000037,0.000754,-0.000334,-0.000034,...,0.000233,0.000031,0.000164,-0.000030,-0.000507,0.000333,-0.000319,0.000038,0.000788,-0.000204
3,28.449265,76.832916,-0.000011,0.000126,0.000222,0.000111,-0.000018,0.000250,-0.000520,0.000099,...,0.000233,-0.000101,0.000386,0.000400,0.000185,0.000932,-0.000656,0.000335,0.000288,-0.000068
4,28.462866,76.832916,-0.000550,0.000672,0.000265,0.000203,-0.000290,0.000163,0.000080,0.000302,...,-0.000087,-0.000799,-0.000402,0.000204,0.000091,0.000342,0.000064,-0.000149,0.000131,-0.000380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,28.830106,77.328716,-0.000180,-0.000210,0.000018,0.000300,0.000281,0.000841,0.000657,-0.000093,...,-0.000277,-0.000015,0.000067,0.000291,0.000789,-0.000107,-0.001590,0.000360,0.000007,-0.000070
1364,28.843708,77.328716,-0.000996,0.000291,-0.000195,-0.000521,0.000729,0.000030,0.000657,-0.000188,...,-0.000349,-0.000120,0.000178,0.000137,0.000789,-0.000431,0.000777,-0.000082,-0.000112,-0.000148
1365,28.857309,77.328716,-0.000180,-0.000792,0.042543,0.000061,0.000232,0.000181,0.000657,0.000117,...,-0.000102,-0.000122,0.000103,-0.000112,-0.000110,0.000347,0.000122,-0.000951,0.000049,-0.000298
1366,28.870910,77.328716,-0.000180,-0.000083,-0.000477,0.000380,0.000204,0.000764,0.000657,-0.000340,...,-0.000070,0.000198,-0.000025,-0.001119,0.000006,0.000100,-0.000155,0.000233,-0.000111,0.000141


In [7]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_So2/Chosen_state_2023_sep_merge_sentinel5p_So2/WB_sep_So2.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_So2/After_krigging_2023_sep_sentinal5p_So2/Wb_2023_sep_So2_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.0170321604509555e-08
Nugget: 1.4777100914233866e-07 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.3345500358636515e-08
Nugget: 6.613535146121916e-08 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 3.232638717730034e-07
Nugget: 1.1652341561609897e-07 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 5.085118894770292e-09
Nugget: 1.0720454674885686e-07 


In [8]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-26,2023-09-27,2023-09-28,2023-09-29
0,21.539446,85.826363,-0.000140,-0.000316,0.000005,-0.000271,0.001138,-0.000397,0.000522,-0.000551,...,0.000110,-0.000135,0.000880,-0.001649,-0.000328,0.000230,0.000408,0.000327,-0.000071,0.000091
1,21.660331,85.826363,-0.000084,-0.000090,-0.000214,0.000602,-0.000501,-0.000049,0.000070,-0.000229,...,0.000252,-0.000675,-0.000297,0.000187,-0.000044,-0.000249,0.000239,-0.000278,0.000430,0.000412
2,21.781215,85.826363,-0.000084,-0.000210,-0.000013,0.000108,-0.000310,-0.000032,0.000020,-0.000192,...,0.000087,0.000059,0.000454,0.000125,0.000143,0.000910,0.000046,-0.000737,-0.000080,-0.000155
3,21.902100,85.826363,-0.000050,-0.000326,0.000182,-0.000329,0.000230,0.000329,-0.000165,0.000486,...,0.000082,-0.000214,-0.000194,-0.002253,-0.000200,0.000309,-0.000059,-0.000074,0.000584,-0.000034
4,22.022985,85.826363,-0.000084,-0.000256,-0.000013,0.000869,-0.000233,-0.000032,0.000020,-0.000633,...,0.001020,0.000201,0.000594,0.000507,-0.000027,0.000659,0.000007,-0.000221,0.000419,0.000063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627,26.737492,89.786363,0.000116,0.000051,-0.000015,-0.000378,-0.000513,-0.000103,0.000104,0.000233,...,0.000147,0.000216,0.000228,0.000552,0.000757,0.000536,0.000449,0.000329,0.000362,0.000163
1628,26.858377,89.786363,0.000261,-0.000224,-0.000015,-0.000007,-0.000076,-0.000037,-0.000072,-0.000046,...,-0.000276,0.000050,-0.000161,-0.000140,0.000423,0.000099,-0.000393,0.000119,0.000709,-0.000068
1629,26.979262,89.786363,-0.000021,-0.000022,-0.000176,-0.000221,0.000040,-0.000103,-0.000043,-0.000092,...,-0.000033,0.000336,0.000277,0.000046,0.000119,0.000826,-0.000312,0.000126,-0.000033,0.000061
1630,27.100146,89.786363,0.000059,0.000051,-0.000015,-0.000182,0.000044,0.000068,-0.000111,-0.000118,...,0.000128,-0.000513,0.000651,-0.000049,0.000111,-0.000868,-0.000002,0.000310,0.000147,-0.000058


In [10]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_So2/Chosen_state_2023_sep_merge_sentinel5p_So2/karnataka_sep_So2.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_So2/After_krigging_2023_sep_sentinal5p_So2/Karnataka_2023_sep_So2_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.0647692051457115e-07
Nugget: 1.5979318675144e-07 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.2646394819287975e-09
Nugget: 1.3319435953184868e-07 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 9.575874670128406e-09
Nugget: 1.3615649295079022e-07 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 3.482062862795363e-08
Nugget: 3.28388547460279e-07 

Cal

In [11]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-26,2023-09-27,2023-09-28,2023-09-29
0,11.574480,74.054306,-0.000031,0.000110,-0.000008,-0.000102,0.000519,-0.000236,0.000143,0.000606,...,-0.000361,0.000100,0.000101,0.000243,0.000208,0.000163,-0.000079,0.000126,-0.000152,-0.000348
1,11.714901,74.054306,0.000137,0.000566,-0.000023,-0.000478,0.000471,-0.000604,-0.000229,0.000488,...,0.000304,-0.000031,0.001707,-0.000390,0.000160,0.000066,0.000014,0.000429,-0.000449,-0.000190
2,11.855323,74.054306,-0.000599,-0.000173,0.000038,-0.000850,0.000118,0.000074,0.000011,-0.000831,...,-0.000244,0.000202,-0.000026,-0.000166,0.000285,0.000127,0.000193,0.000354,-0.000328,0.000203
3,11.995744,74.054306,-0.000243,0.000246,-0.000008,-0.000055,0.000194,-0.000611,0.000171,0.000633,...,-0.000008,-0.000136,0.000941,0.000223,0.000082,0.000133,0.000948,-0.000055,0.000651,-0.000136
4,12.136165,74.054306,0.000435,-0.000431,-0.000008,-0.004033,-0.000158,0.001097,0.000190,-0.000729,...,0.000289,-0.000518,0.000014,-0.000099,0.000555,0.000350,0.000238,0.000373,0.000013,-0.000119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645,17.893435,78.534306,0.000582,0.000108,0.000345,-0.000102,-0.000400,-0.000400,0.000204,0.000004,...,-0.000020,0.000192,0.000144,-0.003484,0.000276,-0.000326,-0.000102,0.000061,0.000662,0.000422
1646,18.033856,78.534306,-0.000003,-0.000642,0.000022,-0.000194,0.001001,0.000187,0.000147,-0.002046,...,0.000047,0.000058,0.000413,0.000285,-0.000038,0.000088,-0.000075,-0.000051,0.000020,-0.000006
1647,18.174278,78.534306,-0.000362,0.001054,0.001092,0.000258,0.000421,0.000126,0.000387,-0.000313,...,0.000040,-0.000024,0.000143,0.000206,-0.000034,0.000707,0.000336,-0.000193,-0.000121,0.000536
1648,18.314699,78.534306,0.000152,0.000323,0.000022,0.001858,0.000295,0.000172,0.000076,0.000004,...,-0.000992,0.000189,-0.000263,0.000220,0.000451,0.001560,0.000716,-0.000155,-0.000046,-0.000200


In [12]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_So2/Chosen_state_2023_sep_merge_sentinel5p_So2/Orissa_sep_So2.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_So2/After_krigging_2023_sep_sentinal5p_So2/Orissa_2023_sep_So2_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 3.317946153679371e-08
Nugget: 2.7934799018020423e-07 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 4.3224182411631545e-08
Nugget: 9.489470277019013e-08 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 5.045556221066724e-09
Nugget: 1.2652481793672744e-07 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 9.816385413642708e-09
Nugget: 1.5829592080759033e-07 



In [ ]:
result_df